In [1]:
import os
import numpy as np
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle  
from tqdm import tqdm
from joblib import dump
from PIL import Image  

In [2]:
import time
import psutil

In [3]:
# 设置数据路径
DATA_DIR = r"C:\Users\19916\Downloads\Fused\datasets\Fused"  

def load_images_from_folder(folder):
    data = []
    for filename in os.listdir(folder):
        if filename.lower().endswith((".jpg", ".png", ".jpeg")):
            img_path = os.path.join(folder, filename)
            try:
                image = Image.open(img_path).convert("L")  
                image = image.resize((64, 64))  
                data.append(np.array(image).flatten())  
            except Exception as e:
                print(f"无法读取图像 {img_path}，跳过：{e}")
    return data

In [4]:
def load_dataset(set_name):
    real_path = os.path.join(DATA_DIR, set_name, "real")
    fake_path = os.path.join(DATA_DIR, set_name, "fake")

    real_images = load_images_from_folder(real_path)
    fake_images = load_images_from_folder(fake_path)

    X = np.array(real_images + fake_images)
    y = np.array([0] * len(real_images) + [1] * len(fake_images))  # 0: real, 1: fake

    return shuffle(X, y, random_state=42)

In [5]:
# 加载数据
print("📂 正在加载数据集...")
X_train, y_train = load_dataset("Train")
X_val, y_val = load_dataset("Val")
X_test, y_test = load_dataset("Test")

📂 正在加载数据集...


In [6]:
# PCA 降维
print("🧠 正在执行 PCA 降维...")
pca = PCA(n_components=0.95, svd_solver='full')
X_train_pca = pca.fit_transform(X_train)
X_val_pca = pca.transform(X_val)
X_test_pca = pca.transform(X_test)

🧠 正在执行 PCA 降维...


In [7]:
# 标准化
print("📊 正在标准化...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_pca)
X_val_scaled = scaler.transform(X_val_pca)
X_test_scaled = scaler.transform(X_test_pca)

📊 正在标准化...


In [8]:
start_time = time.time()

In [9]:
# 训练 LinearSVC
print("🚀 开始训练 LinearSVC 模型...")
clf = LinearSVC(dual=False, max_iter=10000)
clf.fit(X_train_scaled, y_train)
print("✅ 模型训练完成！")

🚀 开始训练 LinearSVC 模型...
✅ 模型训练完成！


In [10]:
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

Training time: 485.68 seconds


In [11]:
# 保存模型
dump(clf, "linear_svc_model.joblib")
dump(pca, "pca_model.joblib")
dump(scaler, "scaler.joblib")
print("💾 模型已保存：linear_svc_model.joblib")

💾 模型已保存：linear_svc_model.joblib


In [12]:
# 评估验证集
print("Validation set evaluation:")
val_preds = clf.predict(X_val_scaled)
print("Validation accuracy:", accuracy_score(y_val, val_preds))
print("Classification report:\n", classification_report(y_val, val_preds))

Validation set evaluation:
Validation accuracy: 0.6600781768567003
Classification report:
               precision    recall  f1-score   support

           0       0.67      0.64      0.65     13555
           1       0.65      0.68      0.67     13563

    accuracy                           0.66     27118
   macro avg       0.66      0.66      0.66     27118
weighted avg       0.66      0.66      0.66     27118



In [13]:
# 评估测试集
print("Test set evaluation:")
test_preds = clf.predict(X_test_scaled)
print("Test accuracy:", accuracy_score(y_test, test_preds))
print("Classification report:\n", classification_report(y_test, test_preds))

Test set evaluation:
Test accuracy: 0.6345029239766082
Classification report:
               precision    recall  f1-score   support

           0       0.64      0.61      0.62      5431
           1       0.63      0.66      0.64      5513

    accuracy                           0.63     10944
   macro avg       0.63      0.63      0.63     10944
weighted avg       0.63      0.63      0.63     10944



In [14]:
# 记录 CPU 和内存
cpu_percent = psutil.cpu_percent()
virtual_memory = psutil.virtual_memory().used / (1024 ** 3)
print(f"CPU Usage: {cpu_percent}%")
print(f"Memory Usage: {virtual_memory:.2f} GB")

CPU Usage: 13.2%
Memory Usage: 12.02 GB
